# Gdsfactory tutorial

Lets demonstrate some basic functionality of gdsfactory. For a much more
detailed tutorial go to `pp/samples`

[based on phidl
tutorial](https://github.com/amccaugh/phidl/blob/master/phidl/phidl_tutorial_example.py#L35)

In [ ]:
import pp

# Create a blank component (essentially an empty GDS cell with some special features)
c = pp.Component('myComponent')

# Create and add a polygon from separate lists of x points and y points
# (Can also be added like [(x1,y1), (x2,y2), (x3,y3), ... ]
poly1 = c.add_polygon( [(-8,6,7,9), (-6,8,17,5)], layer = 0)


# Create some new geometry from the functions available in the geometry library
t = pp.c.text('Hello!')
r = pp.c.rectangle(size = [5,10], layer = 2)

# Add references to the new geometry to c, our blank device
text1 = c.add_ref(t) # Add the text we created as a reference
text2 = c << t # Using the << operator (identical to add_ref()), add the same geometry a second time
r = c << r # Add the rectangle we created

# Now that the geometry has been added to "c", we can move everything around:
text1.movey(25)
text2.move([5,30])
text2.rotate(45)
r.movex(-15)

pp.qp(c) # quickplot it!
pp.show(c) # show it in klayout

# Smarter movement with **ports**

Any Component can have "Port"s in it which allow you to snap geometry together
like legos.  Below is an example where we write a simple function to make a
rectangular waveguide, assign ports to the ends of the rectangle, and then snap
those rectangles together

In [ ]:

def waveguide(width = 10, height = 1, layer = 0):
    WG = pp.Component('waveguide')
    WG.add_polygon( [(0, 0), (width, 0), (width, height), (0, height)], layer = layer)
    WG.add_port(name = 'W0', midpoint = [0,height/2], width = height, orientation = 180)
    WG.add_port(name = 'E0', midpoint = [width,height/2], width = height, orientation = 0)
    return WG

c = pp.Component('waveguides')

wg1 = c << waveguide(width=6, height = 2.5, layer = 1)
wg2 = c << waveguide(width=11, height = 2.5, layer = 2)
wg3 = c << waveguide(width=15, height = 2.5, layer = 3)
wg2.movey(10).rotate(10)
wg3.movey(20).rotate(15)

# And plot the result!
pp.qp(c, new_window = True)
pp.show(c)

In [ ]:
# Now we can connect everything together using the ports:

# Each waveguide has two ports: 'W0' and 'E0'.  These are arbitrary 
# names defined in our waveguide() function above

# Let's keep wg1 in place on the bottom, and connect the other waveguides to it.
# To do that, on wg2 we'll grab the "W0" port and connect it to the "E0" on wg1:
wg2.connect('W0', wg1.ports['E0'])
# Next, on wg3 let's grab the "W0" port and connect it to the "E0" on wg2:
wg3.connect('W0', wg2.ports['E0'])

# And plot the result!
pp.qp(c, new_window = True)


# Manipulating geometry 1 - Basic movement and rotation

There are several actions we can take to move and rotate the geometry.

These actions include movement, rotation, and reflection.

In [ ]:
c = pp.Component('waveguides')

wg1 = c << waveguide(width=6, height = 2.5, layer = 1)
wg2 = c << waveguide(width=11, height = 2.5, layer = 2)
wg3 = c << waveguide(width=15, height = 2.5, layer = 3)

# Create and add a polygon from separate lists of x points and y points
# e.g. [(x1, x2, x3, ...), (y1, y2, y3, ...)]
poly1 = c.add_polygon( [(8,6,7,9), (6,8,9,5)] )

# Alternatively, create and add a polygon from a list of points
# e.g. [(x1,y1), (x2,y2), (x3,y3), ...] using the same function
poly2 = c.add_polygon( [(0, 0), (1, 1), (1, 3), (-3, 3)] )


wg1.move([10,4]) # Shift the second waveguide we created over by dx = 10, dy = 4
wg2.move(origin = [1,1], destination = [2,2]) # Shift the second waveguide over by dx = 1, dy = 1
wg3.move([1,1], [5,5], axis = 'y') # Shift the third waveguide over by dx = 0, dy = 4 (motion only along y-axis)
poly1.movey(4) # Same as specifying axis='y' in the move() command
poly2.movex(4) # Same as specifying axis='x'' in the move() command
wg3.movex(30,40) # Moves "from" x=30 "to" x=40 (e.g. shifts wg3 by +10 in the x-direction)

# wg1.rotate(45) # Rotate the first waveguide by 45 degrees around (0,0)
# wg2.rotate(30, center = [1,1]) # Rotate the second waveguide by 30 degrees around (1,1)
# wg1.reflect(p1 = [1,1], p2 = [1,3]) # Reflects wg3 across the line formed by p1 and p2

pp.qp(c)

# Adding ports

Although our waveguides wg1/wg2/wg3 have ports, they're only references
of the device ``D`` we're working in, and D itself does not -- it only draws
the subports (ports of wg1, wg2, wg3) as a convenience.  We need to add ports
that we specifically want in our new device ``D``. add_port() can take a
port argument which allows you to pass it an underlying reference port to
copy. You can also rename the port if you desire:

In [ ]:

c = pp.Component('waveguides')

wg1 = c << waveguide(width=6, height = 2.5, layer = 1)
wg2 = c << waveguide(width=11, height = 2.5, layer = 2)
wg3 = c << waveguide(width=15, height = 2.5, layer = 3)

wg2.connect('W0', wg1.ports['E0'])
wg3.connect('W0', wg2.ports['E0'])

p1 = c.add_port(port = wg1.ports['W0'], name = "W0")
p2 = c.add_port(port = wg3.ports['E0'], name = "E0")
pp.qp(c)


# Adding references

Now that we have our device ``c`` which is a multi-waveguide device, we
can add references to that device in a new blank canvas we'll call ``c2``.
We'll add two copies of ``c`` to c2, and shift one so we can see them both

In [ ]:
c2 = pp.Component('MultiMultiWaveguide')
mwg1 = c2.add_ref(c)
mwg2 = c2.add_ref(c)
mwg2.move(destination = [10,10])

pp.qp(c2) # quickplot it!

# Like before, let's connect mwg1 and mwg2 together
mwg1.connect(port = "E0", destination = mwg2.ports["W0"])

pp.qp(c2) # quickplot it!


# Labeling

We can also label (annotate) our devices, in order to record information
directly into the final GDS file without putting any extra geometry onto any
layer
This label will display in a GDS viewer, but will not be rendered or printed
like the polygons created by the text().
You can use for example pp.show() to see the labels in Klayout

In [ ]:

c2.add_label(text = 'First label', position = mwg1.center)
c2.add_label('Second label', mwg2.center)

# It's very useful for recording information about the devices or layout
c2.add_label(text = f'The x size of this\nlayout is {c2.xsize}',
            position = (c2.xmax, c2.ymax), layer = 255)

# Again, note we have to write the GDS for it to be visible (view in KLayout)
c2.write_gds('MultiMultiWaveguideWithLabels.gds')
pp.qp(c2) # quickplot does not show the labels
pp.show(c2) # klayout will show the labels



# Making boolean shapes

If you want to subtract one shape from another, merge two shapes, or
perform an XOR on them, you can do that with the `pp.pg.boolean()` function.


The ``operation`` argument should be {not, and, or, xor, 'A-B', 'B-A', 'A+B'}.
Note that 'A+B' is equivalent to 'or', 'A-B' is equivalent to 'not', and
'B-A' is equivalent to 'not' with the operands switched


In [ ]:
import pp
e1 = pp.c.ellipse()
e2 = pp.c.ellipse(radii=(10, 6)).movex(2)
e3 = pp.c.ellipse(radii=(10, 4)).movex(5)
pp.qp([e1, e2, e3])

In [ ]:
c = pp.boolean(A = [e1, e3], B = e2, operation = 'A-B')
pp.qp(c)
pp.show(c)